# Imports

In [86]:
import sqlite3
import pandas as pd
import requests
import json
import numpy as np

In [70]:
conn = sqlite3.connect('./database.sqlite')
cur = conn.cursor()

query = """
        SELECT *
        FROM Teams
        JOIN Unique_Teams ON Teams.TeamName = Unique_Teams.TeamName
        JOIN Teams_in_Matches ON Unique_Teams.Unique_Team_ID = Teams_in_Matches.Unique_Team_ID
        JOIN Matches ON Teams_in_Matches.Match_ID = Matches.Match_ID
        WHERE Matches.Season = 2011 AND Teams.Season = 2011
        """
# data = cur.execute(query).fetchmany(5)
df = pd.DataFrame(cur.execute(query).fetchall())
df.columns = [x[0] for x in cur.description]
df[df.duplicated(subset='Match_ID',keep='first')]

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity,TeamName,Unique_Team_ID,...,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
1,2006,Bayern Munich,29,26,15,166980000,5760000,75000,Bayern Munich,1,...,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
2,2007,Bayern Munich,28,25,12,222930000,7960000,75000,Bayern Munich,1,...,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
3,2008,Bayern Munich,23,26,12,238900000,10390000,75000,Bayern Munich,1,...,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
4,2009,Bayern Munich,29,25,15,283250000,9770000,75000,Bayern Munich,1,...,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
5,2010,Bayern Munich,31,25,17,284500000,9180000,75000,Bayern Munich,1,...,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13799,2012,Erzgebirge Aue,27,25,5,14400000,533000,47728,Erzgebirge Aue,39,...,39,1703,D2,2011,2011-09-16,Erzgebirge Aue,Fortuna Dusseldorf,2,4,A
13800,2013,Erzgebirge Aue,32,26,9,13350000,417000,47728,Erzgebirge Aue,39,...,39,1703,D2,2011,2011-09-16,Erzgebirge Aue,Fortuna Dusseldorf,2,4,A
13801,2014,Erzgebirge Aue,37,24,12,13400000,362000,47728,Erzgebirge Aue,39,...,39,1703,D2,2011,2011-09-16,Erzgebirge Aue,Fortuna Dusseldorf,2,4,A
13802,2016,Erzgebirge Aue,30,25,6,8480000,283000,47728,Erzgebirge Aue,39,...,39,1703,D2,2011,2011-09-16,Erzgebirge Aue,Fortuna Dusseldorf,2,4,A


# DarkSky API

In [61]:
apikey = dict(json.load(open('/home/antoine/.apikeys')))['Darksky']
#Berlin:
lat = 52.5065133
long = 13.1445703
url='https://api.darksky.net/forecast/{apikey}/{lat},{long},{time}?exclude=currently,minutely,flags'
weather = requests.get(url) # 400

<Response [400]>

In [196]:
query = """
        SELECT *
        FROM Matches
        """
# JOIN Teams_in_Matches ON Matches.Match_ID = Teams_in_Matches

df = pd.DataFrame(cur.execute(query).fetchall())
df.columns = [x[0] for x in cur.description]
s11 = df[df['Season']==2011]
s11germ = s11[s11['Div']!='E0']
lat = 52.5065133
long = 13.1445703
weather = []
for date in s11germ['Date'][:5]:
#     print(date)
    time = date
    url='https://api.darksky.net/forecast/{}/{},{},{}T16:00:00'.format(apikey,lat,long,time)
#     print(url)
    response = requests.get(url)
    weather.append(response.text)

weather

#         JOIN Teams_in_Matches ON Matches.Match_ID = Teams_in_Matches.Match_ID
#         JOIN Unique_Teams ON Teams_in_Matches.Unique_Team_ID = Unique_Teams.Unique_Team_ID
#         JOIN Teams ON Unique_Teams.TeamName = Teams.TeamName
#         WHERE Matches.Season = 2011

['{"latitude":52.5065133,"longitude":13.1445703,"timezone":"Europe/Berlin","currently":{"time":1333202400,"summary":"Partly Cloudy","icon":"partly-cloudy-day","precipIntensity":0,"precipProbability":0,"temperature":39.48,"apparentTemperature":31.83,"dewPoint":33.54,"humidity":0.79,"windSpeed":13,"windGust":19.52,"windBearing":330,"cloudCover":0.44,"uvIndex":2,"visibility":5.631},"hourly":{"summary":"Drizzle in the morning and afternoon.","icon":"rain","data":[{"time":1333144800,"summary":"Mostly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":46.36,"apparentTemperature":39.99,"dewPoint":40.77,"humidity":0.81,"windSpeed":14.96,"windGust":26.06,"windBearing":292,"cloudCover":0.75,"uvIndex":0,"visibility":6.216},{"time":1333148400,"summary":"Mostly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":46.36,"apparentTemperature":39.16,"dewPoint":40.86,"humidity":0.81,"windSpeed":18.42,"windGust":27.68,"win

In [182]:
weather

[<Response [400]>,
 <Response [400]>,
 <Response [400]>,
 <Response [400]>,
 <Response [400]>]